<h1 align="center">GPU - PRIMERA ENTREGA</h1>
<h3 align="center">Julian David Alfonso Moreno</h3>

---

## ¿Que es?
<p align="left">Una unidad de procesamiento gráfico o GPU es un coprocesador dedicado al procesamiento de gráficos u operaciones de coma flotante, para aligerar la carga de trabajo del procesador central<p>
<img width="300px" align="left" src="https://upload.wikimedia.org/wikipedia/commons/4/44/6600GT_GPU.jpg"/>

### Importamos las librerias

In [1]:
import numpy as np #para crear vectores y matrices grandes multidimensionales, funciones matemáticas de alto nivel
import os #propiedades del sistema operativo
import gzip #crea y extrae ficheros comprimidos

### Creamos la función  ````load_mnist(ruta, tipo='train')````

##### Lee los datos de la ruta especificada y los guarda en las variables etiquetas 

In [2]:
def load_mnist(ruta, tipo='train'):
    
    #cargamos las rutas(ubicaciones) de las categorias y las imagenes
    ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
    ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
    
    #abrimos el archivo de categorias(labels) en lectura binaria
    with gzip.open(ruta_categorias, 'rb') as rut_cat:
        #guardamos informacion en la variable en etiquetas
        etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)
    
    #abrimos el archivo de imagenes en lectura binaria
    with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
        #guardamos informacion en la variable en imagenes
        imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas),784)
        
    #devolvemos la informacion leida
    return imagenes, etiquetas

## Acceso a Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta = 'gdrive/My Drive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = load_mnist(ruta, tipo='train')
X_test, Y_test = load_mnist(ruta, tipo='test')

Mounted at /content/gdrive


## Reshape de los datos para garantizar que sean matrices no vectores
Diferencia: Un vector es un array **unidimensional** de números. | Una matriz es un array **bidimensional** de números.

In [4]:
#Reajustamos el tamano del vector de 60mil a 59904 y de 10mil a 9984
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

#cada dato será una imagen de 28x28 pixeles
X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

## Importamos TensorFlow2 con keras

In [5]:
%tensorflow_version 2.x   # Para garantizar que la versión 2.x sea importada
import tensorflow as tf
print('Versión de TensorFlow: ' + tf.__version__) #comprobamos la versión

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x   # Para garantizar que la versión 2.x sea importada`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
Versión de TensorFlow: 2.6.0


# Creación del Modelo

In [6]:
tf.random.set_seed(200) #creamos una semilla para hacer random de 0 a 200
model = tf.keras.models.Sequential() #contenedor del modelo

#creamos las 3 capas convolucionales
#filtro convolucionales = 64
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

#filtro convolucionales = 64
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

#filtro convolucionales = 64
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary() #imprimimos en pantalla

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0

## Verificamos disponibilidad de GPU

In [7]:
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))

GPU encontrada: /device:GPU:0


## Entrenamiento con GPU

In [8]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [9]:
import timeit

def entrenamiento_cpu():
  with tf.device('/gpu:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)

  return None

cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')

Epoch 1/2
468/468 [==============================] - 52s 42ms/step - loss: 0.6864 - accuracy: 0.7811 - val_loss: 0.3988 - val_accuracy: 0.8599
Epoch 2/2
468/468 [==============================] - 19s 40ms/step - loss: 0.3938 - accuracy: 0.8609 - val_loss: 0.3411 - val_accuracy: 0.8812


In [10]:
print('Tiempo de entrenamiento: ' + str(cpu_time) + ' segundos')

Tiempo de entrenamiento: 83.00992415499996 segundos
